In [1]:
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import load_model

C:\Users\user\AppData\Local\Temp\ipykernel_51588\119768719.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# 음성데이터 argumentation을 통해서 데이터 수 늘리기 및 오버피팅 방지
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data):
    return librosa.effects.time_stretch(data, rate=0.8)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitchs(data, sampling_rate, n_steps):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=n_steps)


In [3]:
def extract_features(data, sampling_rate):

    result = np.array([])

    # ZCR
    # zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    # result=np.hstack((result, zcr)) # stacking horizontally
    
    # # Chroma_stft
    # stft = np.abs(librosa.stft(data))
    # chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T, axis=0)
    # result = np.hstack((result, chroma_stft)) # stacking horizontally
    
    # # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # # Root Mean Square Value
    # rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    # result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result


In [4]:
# 음성의 특성을 추출한 데이터를 축적하는 함수 (Argumentation된 데이터도 같이)
def get_features(path):
    # duration과 offset은 각 오디오 파일의 시작과 끝에서 오디오가 없는 것을 처리
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    sample_rate = 44100
    # 원래데이터
    res1 = extract_features(data, sample_rate)
    result = np.array(res1)
#     # 노이즈가 추가된 데이터
    noise_data = noise(data)
    res2 = extract_features(noise_data, sample_rate)
    result = np.vstack((result, res2)) # 병렬적으로 추가

#     # 피칭및 스트레칭된 데이터
    new_data = stretch(data)
    data_stretch_pitch = pitchs(new_data, sample_rate, 0.7)
    res3 = extract_features(data_stretch_pitch, sample_rate)
    result = np.vstack((result, res3)) # 병렬적으로 추가

    return result

In [5]:
X, Y = [], []
feature = get_features("soundData/test/test.wav")
for ele in feature:
    X.append(ele)

In [6]:
Features = pd.DataFrame(X)

In [7]:
X = Features.values

In [8]:
X.shape

(3, 148)

In [9]:
scaler = StandardScaler()
x = scaler.fit_transform(X)

In [10]:
x.shape

(3, 148)

In [11]:
x = np.expand_dims(x, axis=2)

In [12]:
model = load_model('sound_classifier_model_MFCC_MEL_44100(version4).h5')

In [13]:
pred_test = model.predict(x)

1/1 [==============================] - 1s 513ms/step


In [14]:
y = np.array(['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness'])

In [15]:
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(y).reshape(-1,1)).toarray()

In [16]:
y_pred = encoder.inverse_transform(pred_test)
df = pd.DataFrame(columns=['Predicted Labels'])
df['Predicted Labels'] = y_pred.flatten()

In [17]:
df['Predicted Labels']

0    neutral
1    neutral
2    disgust
Name: Predicted Labels, dtype: object

In [19]:
y_pred

array([['neutral'],
       ['neutral'],
       ['disgust']], dtype='<U9')